In [4]:
import brightway2 as bw
import numpy as np
import pandas as pd
from copy import copy

# Setup the project

### TODO -> choose project

In [5]:
project = 'GSA for ecoinvent'

In [6]:
if project not in bw.projects:
    bw.projects.create_project(project)
bw.projects.set_current(project)

# Import older versions of databases

### TODO -> give paths to databases files
for now let's use ecoinvent 3.3 cutoff and EXIOBASE 2.2

In [7]:
# Older databases
ei33_path = '/Users/akim/Documents/LCA files/ecoinvent 3.3 cutoff/datasets'  
ex22_path = '/Users/akim/Documents/LCA files/exiobase 2.2/mrIOT_IxI_fpa_coefficient_version2.2.2'
ag12_path = '/Users/akim/Documents/LCA files/agribalyse 1.2/AGRIBALYSEv1.2_Complete.xml'

# Newer databases
ei36_path = '/Users/akim/Documents/LCA files/ecoinvent 3.6 cutoff/datasets'  
# ex34_path = '/Users/akim/Documents/LCA files/exiobase 3.4/IOT_2011_pxp/' #TODO
ag13_path = '/Users/akim/Documents/LCA files/agribalyse 1.3/Agribalyse CSV FINAL_no links_Nov2016v3.CSV'

In [8]:
bw.databases

Databases dictionary with 6 object(s):
	Agribalyse 1.2 - ecoinvent 3.3 cutoff
	Agribalyse 1.2 - ecoinvent 3.3 cutoff - new biosphere
	EXIOBASE 2.2
	biosphere3
	ecoinvent 3.3 cutoff
	ecoinvent 3.6 cutoff

## 1. Biosphere3

In [9]:
bw.bw2setup()

Biosphere database already present!!! No setup is needed


## 2. Ecoinvent 3.3

In [10]:
ei33_name = "ecoinvent 3.3 cutoff"  
if ei33_name in bw.databases:
    print(ei33_name + " database already present!!! No import is needed")
else:
    ei33 = bw.SingleOutputEcospold2Importer(ei33_path, ei33_name)
    ei33.apply_strategies()
    ei33.match_database(db_name='biosphere3',fields=('name', 'category', 'unit', 'location'))
    ei33.statistics()

ecoinvent 3.3 cutoff database already present!!! No import is needed


In [11]:
if ei33_name in bw.databases:
    print(ei33_name + " database already present!!! No import is needed")
else:
    ei33.write_database()

ecoinvent 3.3 cutoff database already present!!! No import is needed


## 3. Exiobase 2.2

In [12]:
# 2. Exiobase 
from bw2io.importers.exiobase import Exiobase22Importer

In [13]:
ex22_name = 'EXIOBASE 2.2'
if ex22_name in bw.databases:
    print(ex22_name + " database already present!!! No import is needed")
else:
    ex22 = Exiobase22Importer(ex22_path, ex22_name)
    ex22.apply_strategies()

EXIOBASE 2.2 database already present!!! No import is needed


In [14]:
if ex22_name in bw.databases:
    print(ex22_name + " database already present!!! No import is needed")
else:
    ex22.write_database()

EXIOBASE 2.2 database already present!!! No import is needed


## 4. Agribalyse 1.2
better see: [how to import](https://nbviewer.jupyter.org/urls/bitbucket.org/cmutel/brightway2/raw/default/notebooks/IO%20-%20Importing%20Agribalyse%20with%20Ecoinvent%202.2.ipynb)

In [15]:
def import_agribalyse12(ag12_path, ag12_name, ei_name):

    ag12_ei_name = ag12_name + ' - ' + ei_name
    
    if ei_name not in bw.databases:
        print('Cannot find database: ' + ei_name)
        return
    
    if ag12_ei_name in bw.databases:
        print(ag12_ei_name + " database already present!!! No import is needed")
    else:
        ag12_ei = bw.SingleOutputEcospold1Importer(ag12_path, ag12_ei_name)
        ag12_ei.apply_strategies()
        ag12_ei.statistics()
        
        ag12_ei.write_excel(True)
        from bw2io.strategies.simapro import normalize_simapro_biosphere_categories, normalize_simapro_biosphere_names
        ag12_ei.apply_strategy(normalize_simapro_biosphere_categories)
        ag12_ei.apply_strategy(normalize_simapro_biosphere_names)

        from bw2io.strategies import link_iterable_by_fields
        import functools
        ag12_ei.apply_strategy(functools.partial(link_iterable_by_fields, other=bw.Database("biosphere3"), kind="biosphere"))
        ag12_ei.statistics()

        ag12_ei.write_excel(True)
        
        ag12_ei_new_biosphere_name = ag12_ei_name + " - new biosphere"
        bw.Database(ag12_ei_new_biosphere_name).register()
        ag12_ei.add_unlinked_flows_to_biosphere_database(ag12_ei_new_biosphere_name)

        def get_unlinked(data):
            for ds in ag12_ei.data:
                for exc in ds['exchanges']:
                    if exc['type'] == 'production' and exc['name'] == 'Alfalfa, conventional, for animal feeding, at farm gate':
                        return ds, exc

        ds, exc = get_unlinked(ag12_ei.data)

        from bw2io.strategies import link_technosphere_based_on_name_unit_location
        ag12_ei.apply_strategy(link_technosphere_based_on_name_unit_location)
        ag12_ei.statistics()

        def get_unlinked(data):
            for ds in ag12_ei.data:
                for exc in ds['exchanges']:
                    if exc['type'] == 'production' and not exc.get('input'):
                        return ds, exc

        ds, exc = get_unlinked(ag12_ei.data)

        for ds in ag12_ei.data:
            for exc in ds['exchanges']:
                if exc['type'] == 'production' and not exc.get('input'):
                    print("Fixing:", exc['name'])
                    exc['type'] = 'technosphere'

        ag12_ei.apply_strategy(link_technosphere_based_on_name_unit_location)
        ei_name_short = ei_name #TODO
        ag12_ei.apply_strategy(functools.partial(link_technosphere_based_on_name_unit_location, external_db_name=ei_name_short))
        ag12_ei.statistics()

        ag12_ei.add_unlinked_activities()
        ag12_ei.statistics()

        for ds in ag12_ei.data:
            del ds['code']

        from bw2io.strategies import set_code_by_activity_hash
        ag12_ei.apply_strategy(set_code_by_activity_hash)

        ag12_ei.apply_strategy(functools.partial(
                link_iterable_by_fields,
                other=ag12_ei.data,
                fields=('name', 'location', 'unit'),
                relink=True
        ))
        ag12_ei.statistics()
        
        if ag12_ei_name in bw.databases:
            print(ag12_ei_name + " database already present!!! No import is needed")
        else:
            ag12_ei.write_database()

In [16]:
ag12_name = 'Agribalyse 1.2'
import_agribalyse12(ag12_path, ag12_name, ei33_name)

Agribalyse 1.2 - ecoinvent 3.3 cutoff database already present!!! No import is needed


In [17]:
bw.databases

Databases dictionary with 6 object(s):
	Agribalyse 1.2 - ecoinvent 3.3 cutoff
	Agribalyse 1.2 - ecoinvent 3.3 cutoff - new biosphere
	EXIOBASE 2.2
	biosphere3
	ecoinvent 3.3 cutoff
	ecoinvent 3.6 cutoff

## 5. HEIA

### TODO -> ask Andi for files

# Import newer versions of databases

## 1. Ecoinvent 3.6 cutoff

In [18]:
ei36_name = "ecoinvent 3.6 cutoff"
if ei36_name in bw.databases:
    print(ei36_name + " database already present!!! No import is needed")
else:
    ei36 = bw.SingleOutputEcospold2Importer(ei36_path, ei36_name)
    ei36.apply_strategies()
    ei36.match_database(db_name='biosphere3',fields=('name', 'category', 'unit', 'location'))
    ei36.statistics()

ecoinvent 3.6 cutoff database already present!!! No import is needed


In [19]:
if ei36_name in bw.databases:
    print(ei36_name + " database already present!!! No import is needed")
else:
    ei36.write_database()

ecoinvent 3.6 cutoff database already present!!! No import is needed


In [20]:
bw.databases

Databases dictionary with 6 object(s):
	Agribalyse 1.2 - ecoinvent 3.3 cutoff
	Agribalyse 1.2 - ecoinvent 3.3 cutoff - new biosphere
	EXIOBASE 2.2
	biosphere3
	ecoinvent 3.3 cutoff
	ecoinvent 3.6 cutoff

# --------------------------------------- In progress ---------------------------------------

## 2. Agribalyse 1.3 - NOT DONE YET

In [21]:
ei_name = ei36_name

In [22]:
ag13_name = 'Agribalyse 1.3'
ag13_ei_name = ag13_name + ' - ' + ei_name
if ag13_ei_name in bw.databases:
    print(ag13_ei_name + " database already present!!! No import is needed")
else:
    ag13_ei = bw.SimaProCSVImporter(ag13_path, ag13_ei_name)
    ag13_ei.apply_strategies()
    ag13_ei.statistics()

Extracted 1189 unallocated datasets in 3.11 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: assign_only_product_as_production
Applying strategy: drop_unspecified_subcategories
Applying strategy: sp_allocate_products
Applying strategy: fix_zero_allocation_products
Applying strategy: split_simapro_name_geo
Applying strategy: strip_biosphere_exc_locations
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: set_code_by_activity_hash
Applying strategy: link_technosphere_based_on_name_unit_location
Applying strategy: change_electricity_unit_mj_to_kwh
Applying strategy: set_lognormal_loc_value_uncertainty_safe
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_simapro_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: normalize_simapro_biosphere_names
Applying strategy: migrate_exchanges
Applying strategy: fix_localized_water_flows
A

In [23]:
simapro_ei_migration = 'simapro-' + ei_name.split(' ')[0] + '-' + ei_name.split(' ')[1]
ag13_ei.migrate(simapro_ei_migration)
ag13_ei.match_database(ei_name, fields=('reference product', 'location', 'unit', 'name'))

Applying strategy: link_iterable_by_fields


/Users/akim/miniconda3/envs/gsa-ei/lib/python3.7/site-packages/bw2io/importers/base_lci.py:384: UserWarning: Skipping migration simapro-ecoinvent-3.6 because it isn't installed.
  "Skipping migration {} because it isn't installed.".format(migration_name)


In [24]:
ag13_ei.statistics()

1261 datasets
115259 exchanges
8805 unlinked exchanges
  Type biosphere: 436 unique unlinked exchanges
  Type technosphere: 439 unique unlinked exchanges


(1261, 115259, 8805)

In [25]:
ag13_ei.write_excel(True)

Wrote matching file to:
/Users/akim/Library/Application Support/Brightway3/GSA-for-ecoinvent.34cad902cfa956f8f1e5e24776ceebde/output/db-matching-Agribalyse-13-ecoinvent-36-cutoff-unlinked.xlsx


In [26]:
ag13_ei_new_biosphere_name = ag13_ei_name + " - new biosphere"
bw.Database(ag13_ei_new_biosphere_name).register()
ag13_ei.add_unlinked_flows_to_biosphere_database(ag13_ei_new_biosphere_name)

OperationalError: database is locked

In [ ]:
ag13_ei.statistics()

In [ ]:
list(ag13_ei.unlinked)

In [ ]:
agribalyse_transport_data = {
    'fields': ['name', 'location'],
    'data': [
        (
            ('market for transport, freight, lorry 16-32 metric ton, EURO3', 'GLO'), 
            {
                'location': 'RoW'
            }
        )
    ]
}

bw.Migration("agribalyse-transport-freight-lorry").write(
    agribalyse_transport_data,
    description="Fix some transport locations from GLO to RoW"
)

In [ ]:
ag13_ei.migrate("agribalyse-transport-freight-lorry")

In [ ]:
ag13_ei.statistics()

In [ ]:
ag13_ei.match_database(ei_name, fields=('location', 'unit', 'name'))

In [ ]:
ag13_ei.statistics()

In [ ]:
ag13_ei.add_unlinked_activities()
ag13_ei.statistics()

In [ ]:
for a in list(ag13_ei.unlinked):
    print(a['name'])

In [ ]:
a

In [ ]:
db = bw.Database('ecoinvent 3.6 cutoff')

In [ ]:
[act for act in db if act['name'] == a['name'] and act['location']==a['location']]

In [ ]:
[act for act in db if 'market for transport, freight, lorry 16-32 metric ton' in act['name']]

# Playground

# Create ecoinvent and exiobase LCA objects

In [ ]:
n_methods = 2
methods = {_: bw.methods.random() for _ in range(n_methods)}
methods

In [ ]:
# 1. Ecoinvent
ei_demand = {bw.Database(ei_name).random(): 1}
ei_lca = bw.LCA(ei_demand, methods[0])
ei_lca.lci()
ei_lca.lcia()
ei_lca.score

In [ ]:
# 2. Exiobase
ex_demand = {bw.Database(ex_name).random(): 1}
ex_lca = bw.LCA(ex_demand, methods[0])
ex_lca.lci()
ex_lca.lcia()
ex_lca.score

In [ ]:
methods

# Consumption model, Andi's code

In [ ]:
methods

In [ ]:
from consumption_copy import ConsumptionLCA

In [ ]:
# consumption_model_path = 'Draft_Paper_8_v11_SupportingInformation.xlsx'
# sheet_lca_modeling = 'Overview & LCA-Modeling'

consumption_model_path = 'Consumption_Cockpit_adj.xlsx'
sheet_lca_modeling = 'LCA-Modelling'

In [ ]:
excel = {
    'path': consumption_model_path,
    'sheet': sheet_lca_modeling
}

In [ ]:
a = ConsumptionLCA(ei_lca, ex_lca, methods, excel, archetypes=True)